## The SageMath code for [1], Section 3.5

In our work [1], we proved that the period polynomials attached to the derivative of 
$L$-functions of cusp forms $f \in S_k(\Gamma_0(N))$ has all of its zeros in the unit circle, for all but finitely many pairs of weight $k$ and level $N$. We tried to cover (some of) the remaining finitely many cusp forms. 

The list of the pair of weight and level $(k, N)$ which requires the manual verifications can be found in [1], Table 3. We wrote SageMath codes to do this. Although still finitely many cusp forms are remaining to be verified, we could reduce the Table 3 into much simpler table, Table 1. 

Our codes consist of two parts. This file, "01_L_vales.ipynb", generates the L'-values attached to cusp newforms.

Based on this values, "02_number_of_zeros_of_period_polynomial.ipynb" generates the period polynomials and counts the number of zeros on the unit circle of them.

All of the codes are written by Hojin Kim, one of the authors of [1]. 

--- 

Reference(s) : 

[1] Im, Bo-Hae; Kim, Hojin; Riemann hypothesis for period polynomials attached to the derivatives of $L$-functions of cusp forms for $Γ_0(N)$. _J. Math. Anal. Appl._ 509 (2022), no. 2, Paper No. 125971.

In [2]:
import numpy as np 
import pandas as pd
import pickle

In [3]:
# Table 3 in [1] gives the pair of $m = (k-2)/2$ and $N_m$, such that the statement holds for 
#     all $N >= N_m$ for given weight $k$. 
# tab, the python dictionary variable, is this Table 3, i.e. tab[3] = 1855 since $N_3 = 1855$. 
# We have to verify the statement for level $N$ with  $1 <= N < N_m$ when the level $k$ is given. 
#
# Note that $N_m =1$ for $m > 538$ or $m=1,2$; thus, tab contains all of the informations 
#     of (finitely many) cusp forms to verify. 

tab = {k:v for k, v in [(3, 1855), (4, 564), (5, 226), (6,123), (7,73), (8, 48), 
                        (9, 33), (10, 24), (11, 19), (12,15), (13,12), (14,10), 
                        (15,8), (16,7), (17,6), (18,6), (19,5), (20,5)]}

for m in range(21, 30):
    tab[m] = 4
for m in range(30, 74):
    tab[m] = 3
for m in range(74, 538):
    tab[m] = 2

In [5]:
# This cell generates and save the values of $L'$ values as pickle file. 
# This code makes one pickle files for one weight. 
# Each pickle file consists of lists with information of the corresponding cusp form 
#     and their $L'$-values at 1, 2, ..., k-1, where $k$ is weight;
#     in detail, it is an array, that each elements are again an 
#     array of [weight, level, idx, eps, df], where 
#       - weight and level are the weight and level of the cusp form, 
#       - idx is the index of cusp form on the list of cuspforms 
#             generated by SageMath function CuspForms, 
#       - eps is the epsilon value. This value is 1 or -1 and satisfies 
#             $Lambda(s) = eps*Lambda(weight-s)$ where Lambda is the completed L-function, 
#       - df is a table(pandas dataframe) of $L'$-values.
#
# The pickle files are saved in "./L_values". 
# You can modify this directory. In that case you have to modify the "02….ipynb" file too.

saving_directory = r"./L_values"

try:
    os.mkdir(saving_directory)
except FileExistsError:
    pass


# The following for loop, if it was written for the complete verification, 
#     would be started as 
#         for m in range(3, 539):
#     to cover all of the remaining finitely many cases.  
# What we have done in our work [1] is the verification for $m = 7, 8, ..., 237$.
# The following code generates, as an example, the $L'$ values of cusp forms
#     for m = 20, 21, ..., 29, i.e. k = 42, 44, ..., 60. 

# Also, note that this code does not support the multiprocessing. 
# If your computing device for SageMath has multiple CPUs, it would be great to 
#     modify this following code for the better performance. 
# The easiest way to do this would be doing it "manually", i.e. copy and paste this 
#     ipynb file and run them individually. For example, if you want to run this 
#     code with 4 CPUs, make the four copies of this code and run the following 
#     for loop with different m's. I.e., running each of them with the following 
#     for loop headers
#         for m in range(20, 30, 4): # It will do for m = 20, 24, 28
#         for m in range(21, 30, 4): # It will do for m = 21, 25, 29
#         for m in range(22, 30, 4): # It will do for m = 22, 26
#         for m in range(23, 30, 4): # It will do for m = 23, 27
#     will generates the pickle files for each weight.
# Or you can modify this code for multiprocessing in itself (in one file).

for m in range(20, 22): # the code for m = 20, 21. 
    form_dict = []
    weight = 2*m+2
    print('weight is {}'.format(weight))
    
    for level in range(1, tab[m]):
        print('_level is {}'.format(level))
        cuspforms = CuspForms(level, weight).newforms(names='x')
        for idx, cuspform in enumerate(cuspforms):
            print('__ {}th form of this weight and level'.format(idx))
            L = cuspform.lseries()
            def LL(z): 
                return L(z) * gamma(z) * (sqrt(level) / (2 * pi))^z
            def LL_d(z) : 
                return L.derivative(z) * gamma(z) * (sqrt(level) / (2 * pi))^z + L(z) * gamma(z)*psi(z) * (sqrt(level) / (2 * pi))^z + L(z) * gamma(z) * (sqrt(level) / (2 * pi))^z * log(sqrt(level)/(2* pi))        
            if LL(1) >0:
                eps = 1 
            else:
                eps = -1
                
            if eps == 1 : 
                half_L_d_values = [LL_d(idx).n() for idx in range(1, weight/2 )]
                L_d_values = half_L_d_values + [0] + [-1 * x for x in half_L_d_values[::-1]]

            elif eps == -1 : 
                half_L_d_values = [LL_d(idx).n() for idx in range(1, weight/2 +1)]
                L_d_values = half_L_d_values + half_L_d_values[::-1][1:]

            df = pd.DataFrame({'L_deriv' : L_d_values}, 
                              index = np.arange(1, weight))
            
            form_dict.append([weight, level, idx, eps, df])
    

    with open('{}/m{}.pkl'.format(saving_directory, m), 'wb') as writing:
        pickle.dump(form_dict, writing)

weight is 42
_level is 1
__ 0th form of this weight and level
verbose -1 (535: dokchitser.py, derivative) Warning: Loss of 2 decimal digits due to cancellation
verbose -1 (488: dokchitser.py, __call__) Warning: Loss of 2 decimal digits due to cancellation
verbose -1 (535: dokchitser.py, derivative) Warning: Loss of 2 decimal digits due to cancellation
verbose -1 (488: dokchitser.py, __call__) Warning: Loss of 2 decimal digits due to cancellation
verbose -1 (535: dokchitser.py, derivative) Warning: Loss of 3 decimal digits due to cancellation
verbose -1 (488: dokchitser.py, __call__) Warning: Loss of 3 decimal digits due to cancellation
verbose -1 (535: dokchitser.py, derivative) Warning: Loss of 3 decimal digits due to cancellation
verbose -1 (488: dokchitser.py, __call__) Warning: Loss of 3 decimal digits due to cancellation
verbose -1 (535: dokchitser.py, derivative) Warning: Loss of 3 decimal digits due to cancellation
verbose -1 (488: dokchitser.py, __call__) Warning: Loss of 3 dec

In [6]:
1+1

2

In [14]:
form_dict[0][-1]['L_deriv']

1     -1.28041905230311e17
2     -1.89164428974933e16
3     -2.86146016531503e15
4     -4.43446359128479e14
5     -7.04458560346465e13
6     -1.14788615705718e13
7     -1.91978287696663e12
8     -3.29768830494670e11
9     -5.82209088487460e10
10    -1.05725700860385e10
11     -1.97626332409358e9
12     -3.80545182009279e8
13     -7.55423956887040e7
14     -1.54699979944506e7
15     -3.26974020153528e6
16       -713332.197197604
17       -160512.976145894
18       -37170.2725484708
19       -8813.69935712520
20       -2115.06386695229
21       -482.774374163306
22                       0
23        482.774374163306
24        2115.06386695229
25        8813.69935712520
26        37170.2725484708
27        160512.976145894
28        713332.197197604
29      3.26974020153528e6
30      1.54699979944506e7
31      7.55423956887040e7
32      3.80545182009279e8
33      1.97626332409358e9
34     1.05725700860385e10
35     5.82209088487460e10
36     3.29768830494670e11
37     1.91978287696663e12
3